<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Script-setup" data-toc-modified-id="Script-setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Script setup</a></span></li><li><span><a href="#Data-directory-preparention" data-toc-modified-id="Data-directory-preparention-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data directory preparention</a></span></li><li><span><a href="#Data-file-preperation" data-toc-modified-id="Data-file-preperation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data file preperation</a></span><ul class="toc-item"><li><span><a href="#input-directory-(all-general-raw-data-files)" data-toc-modified-id="input-directory-(all-general-raw-data-files)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><code>input</code> directory (all general raw data files)</a></span></li><li><span><a href="#processed-directory-(all-data-files-produced-in-this-repository)" data-toc-modified-id="processed-directory-(all-data-files-produced-in-this-repository)-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span><code>processed</code> directory (all data files produced in this repository)</a></span></li></ul></li><li><span><a href="#Load-data-functions" data-toc-modified-id="Load-data-functions-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Load data functions</a></span></li><li><span><a href="#Load-data-sets" data-toc-modified-id="Load-data-sets-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Load data sets</a></span></li><li><span><a href="#ENTSO-E-generation-per-technology-CI" data-toc-modified-id="ENTSO-E-generation-per-technology-CI-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>ENTSO-E generation per technology CI</a></span><ul class="toc-item"><li><span><a href="#EF-preparation" data-toc-modified-id="EF-preparation-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>EF preparation</a></span></li><li><span><a href="#Plots" data-toc-modified-id="Plots-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Plots</a></span></li></ul></li><li><span><a href="#ENTSO-E-Power-Stats-generation-per-technology-CO2-Signal" data-toc-modified-id="ENTSO-E-Power-Stats-generation-per-technology-CO2-Signal-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>ENTSO-E Power Stats generation per technology CO2 Signal</a></span></li><li><span><a href="#Compair-Bottom-Up-and-Top-down" data-toc-modified-id="Compair-Bottom-Up-and-Top-down-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Compair Bottom Up and Top down</a></span></li><li><span><a href="#Plotting-CI" data-toc-modified-id="Plotting-CI-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Plotting CI</a></span></li><li><span><a href="#Export-final-CO2-signals" data-toc-modified-id="Export-final-CO2-signals-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Export final CO2 signals</a></span></li><li><span><a href="#Plottting" data-toc-modified-id="Plottting-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Plottting</a></span></li></ul></div>

# Introduction

This notebook uses the calculated EFs to calculate the CI for EU countries.

# Script setup

In [1]:
import os
import logging

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from IPython.display import Image 
import datetime
from matplotlib.lines import Line2D

#GEO
import geopandas as gpd
#from pyproj import CRS
#from pyproj import Transformer
#from shapely.geometry import Point

%matplotlib inline
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [15, 10]
sns.set_theme()

#helpers
from helpers import change_ENTSOE_ProductionTypeName

# Data directory preparention

Create input, processed and output folders if they don't exist
If the paths are relative, the corresponding folders will be created inside the current working directory.
- input -> all needed input data
- processed -> save point and exchange with other scripts
- output -> final emission factors

In [2]:
input_directory_path = os.path.join('input')
processed_directory_path = 'processed'
output_directory_path = os.path.join('output')

os.makedirs(input_directory_path, exist_ok=True)
os.makedirs(processed_directory_path, exist_ok=True)
os.makedirs(output_directory_path, exist_ok=True)

# Data file preperation

## `input` directory (all general raw data files)

- 1) ENTSO-E production per type
Production per type from ENTSO-E
Original data: ENTSO-E Transparency Platform, Actual Generation per Production Type Available online: 
https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show
Processed with the following script that converts the data to hourly resolution (private script from INATECH):


 File - > time_series_60min_multiindex.csv - > generation per technology
 
- 2) ENTSO-E production per unit
Production per Unit from ENTSO-E
Original data: ENTSO-E Transparency Platform, Actual Generation per Generation Unit Available online: https://transparency.entsoe.eu/generation/r2/actualGenerationPerGenerationUnit/show (accessed on Apr 29, 2020).
Processed with the following script that converts the data to hourly resolution (private script from INATECH):
https://github.com/INATECH-CIG/entso-e_GenerationOutputPerUnit

File - > gen_data.csv -> hourly generation data per unit

In [3]:
# Checks if the the input directories are empty or not
# Checks all filenames in the input directory

if not os.listdir(input_directory_path) :
    print("The directory for the bootom up method is empty. Please provide the data to the directory as described in the instructions above.")


filenames = [os.path.join(input_directory_path, fn) for fn in os.listdir(input_directory_path)]

print(filenames)

['input\\Bootom_up_methode', 'input\\CO2_Signal', 'input\\ember_EF.csv', 'input\\entso.csv', 'input\\entsoe-statistical-factsheet-2018-stacked.csv', 'input\\entsoe_gen_type_hourly_table.csv', 'input\\entsoe_gen_type_hourly_table1.csv', 'input\\entso_gen_type_hourly_table_scaled.csv', 'input\\entso_scaled.csv', 'input\\gen_data_2015.csv', 'input\\gen_data_2016.csv', 'input\\gen_data_2017.csv', 'input\\gen_data_2018.csv', 'input\\Matching', 'input\\MDV_table.csv', 'input\\time_series_60min_multiindex.csv', 'input\\top_down_methode', 'input\\unit_data_2015.csv', 'input\\unit_data_2016.csv', 'input\\unit_data_2017.csv', 'input\\unit_data_2018.csv']


## `processed` directory (all data files produced in this repository)

- 1) EF per type and county

File - > EF_bottom_up_methode.csv

- 2) EF per unit

File -> generation_unit_EF_bottom_up_methode.csv

In [4]:
# Checks if the the input directories are empty or not
# Checks all filenames in the input directory

if not os.listdir(processed_directory_path) :
    print("The directory for the CO2 Signal is empty. Please provide the data to the directory as described in the instructions above.")


filenames = [os.path.join(processed_directory_path, fn) for fn in os.listdir(processed_directory_path)]

print(filenames)

['processed\\CI_bottom_up_methode.csv', 'processed\\CI_MAP_AP_CHP_heat_top_down.csv', 'processed\\CI_MAP_AP_without_heat_top_down.csv', 'processed\\EF_bottom_up_methode.csv', 'processed\\generation_unit_EF_bottom_up_methode.csv', 'processed\\Matching_Entso_EUTL_EU.csv', 'processed\\_CO2_intensity_by_technology.png']


# Load data functions

In [5]:
def load_generation_per_unit(path, fn):
    """
    Load the ENTSO-E gernation per unit data.
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    generation = pd.read_csv(os.path.join(path, fn),sep = ',',index_col=0,parse_dates=True)
    
    return generation

def load_unit_CO2_data(path, fn):
    """
    Own generation unit information with CO2 information per unit
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    df = pd.read_csv(os.path.join(path, fn),sep = ',',index_col=0)
    
    # set name for the index
    #df.index.set_names('eic_g', inplace=True)

    return df

def load_EF(path, fn):
    """
    Own generation EF per tech and country
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    df = pd.read_csv(os.path.join(path, fn),sep = ',',index_col=[0,1])
    
    return df

def load_timeseries_ENTSOE(path, fn):
    """
    Read generation data from time-series package own modification.


    Parameters
    ----------
    path : TYPE
        DESCRIPTION.
    fn : TYPE
        DESCRIPTION.

    Returns
    -------
    generation : TYPE
        DESCRIPTION.

    """
   
    generation = (pd.read_csv(os.path.join(path, fn), index_col=[0], header=[0, 1], parse_dates=True)
                    .dropna(how="all", axis=0))
       
    
    return generation

# Load data sets

#### Load ENTSO-E generation per type

In [6]:
entsoe_gen_type = load_timeseries_ENTSOE(input_directory_path, 'entso_gen_type_hourly_table_scaled.csv')

#### Load EF per technology

In [7]:
EF = load_EF(processed_directory_path, 'EF_bottom_up_methode.csv')

In [8]:
EF.head(10)

EF Verified_EF  EF_round_in_kwh
MapCode ProductionTypeName                                       
AT      gas                 0.360910        True           360.91
        hard_coal           1.069627        True          1069.63
BE      gas                 0.389955        True           389.95
CZ      hard_coal           1.010505        True          1010.50
        lignite             0.932336        True           932.34
DE      gas                 0.409985        True           409.98
        hard_coal           0.909259        True           909.26
        lignite             1.135463        True          1135.46
        other_fossil        1.620500        True          1620.50
DK      gas                 0.456925       False           456.93

# ENTSO-E generation per technology CI

In this section we using the country specific EF for generating the CI for all EU countries. The production data are based on the production per type data from ENTSO-E

In [9]:
# take a look at the generation data
entsoe_gen_type.head(5)

region                         AT                                      \
variable                  biomass        gas   hard_coal        hydro   
utc_timestamp                                                           
2018-01-01 00:00:00+00:00     0.0  85.185998  189.813350  3633.105979   
2018-01-01 01:00:00+00:00     0.0  83.986195  188.547928  3557.076855   
2018-01-01 02:00:00+00:00     0.0  82.786393  188.547928  3439.774779   
2018-01-01 03:00:00+00:00     0.0  86.385801  188.547928  3432.171866   
2018-01-01 04:00:00+00:00     0.0  89.985209  188.547928  3616.814024   

region                                                                     \
variable                  other_fossil other_renewable solar wind_onshore   
utc_timestamp                                                               
2018-01-01 00:00:00+00:00   376.712329             NaN   0.0   767.537636   
2018-01-01 01:00:00+00:00   376.712329             NaN   0.0   854.254629   
2018-01-01 02:00:00+00:00   376.712329             NaN   0.0   638.384669   
2018-01-01 03:00:00+00:00   376.712329             NaN   0.0   417.902102   
2018-01-01 04:00:00+00:00   376.712329             NaN   0.0   296.129304   

region                             BE               ...           SI  \
variable                      biomass          gas  ... wind_onshore   
utc_timestamp                                       ...                
2018-01-01 00:00:00+00:00  310.564338  1486.203046  ...          0.0   
2018-01-01 01:00:00+00:00  205.181412  1551.580523  ...          0.0   
2018-01-01 02:00:00+00:00  178.621922  1538.055173  ...          0.0   
2018-01-01 03:00:00+00:00  175.522424  1501.492141  ...          0.0   
2018-01-01 04:00:00+00:00  175.362106  1515.711098  ...          0.0   

region                            SK                                     \
variable                     biomass         gas  hard_coal       hydro   
utc_timestamp                                                             
2018-01-01 00:00:00+00:00  75.757569  223.815846  71.307423  490.635794   
2018-01-01 01:00:00+00:00  75.488926  220.374486  71.159174  497.114042   
2018-01-01 02:00:00+00:00  75.488926  206.609047  69.231947  552.469700   
2018-01-01 03:00:00+00:00  79.787227  178.823253  67.156471  632.576139   
2018-01-01 04:00:00+00:00  79.249940  182.901901  68.787202  605.737684   

region                                                           \
variable                      lignite      nuclear other_fossil   
utc_timestamp                                                     
2018-01-01 00:00:00+00:00  174.510973  1707.038703   127.939921   
2018-01-01 01:00:00+00:00  172.955443  1704.599138   127.138292   
2018-01-01 02:00:00+00:00  169.844384  1697.092784   125.454872   
2018-01-01 03:00:00+00:00  163.816707  1692.307483   121.740660   
2018-01-01 04:00:00+00:00  165.858340  1697.468101   123.317196   

region                                           
variable                  other_renewable solar  
utc_timestamp                                    
2018-01-01 00:00:00+00:00             0.0   0.0  
2018-01-01 01:00:00+00:00             0.0   0.0  
2018-01-01 02:00:00+00:00             0.0   0.0  
2018-01-01 03:00:00+00:00             0.0   0.0  
2018-01-01 04:00:00+00:00             0.0   0.0  

[5 rows x 369 columns]

## EF preparation 

In [10]:
# Mean EF for all calculated countries and technologies

EF_mean = EF.groupby('ProductionTypeName').mean()
EF_mean.loc['other_fossil'] = 0.75
EF_mean = EF_mean.EF.to_dict()


print(EF_mean)

{'gas': 0.40963087512181773, 'hard_coal': 0.9644313218064282, 'lignite': 1.1948675515652707, 'other_fossil': 0.75}


In [11]:
# If no country-specific EFs are available and also no mean EF is available for a country and technology, we use the following EFs. Source:

EF_default = {'biomass':0,
            'geothermal':0,
            'hydro':0,
            'nuclear':0,
            'oil':0.8,
            'other_renewable':0,
            'solar':0,
            'waste':0.2,
            'wind_offshore':0,
            'wind_onshore':0,
            'marine':0,
            'oil shale':0.51,
            'other_fossil': 0.75}


In [12]:
# Create a dataframe with the EF for each country and technology. If no specific EF is available, we use the default EF .
EF_multi = {}

for i in entsoe_gen_type.columns:
    if i in EF:
        EF_multi[i] = EF[i]
    elif i[1] in EF_mean:
        EF_multi[i] = EF_mean[i[1]]
    else:
        EF_multi[i] = EF_default[i[1]]

In [13]:
# calculate emissions based on EF and generation per type
CO2_emission_entsoe_gen_type = entsoe_gen_type * EF_multi

In [14]:
# From the previously calculated emissions, an emission signal (g CO2 / kWh) can now be calculated for each country.
# Since emissions and generation are available per technology, they must first be grouped and summed. 

# Emission in g CO2
CO2_emission = (CO2_emission_entsoe_gen_type * 1000000).groupby(level=0, axis=1).sum()
# generation in kWh
Generation = (entsoe_gen_type * 1000).groupby(level=0, axis=1).sum()

# CI in g CO2/kWh
Carbon_intensitiy_entsoe_gen_type = CO2_emission.div(Generation)

In [15]:
Carbon_intensitiy_entsoe_gen_type.head(5)

region,AT,BE,BG,CH,CY,CZ,DE,DE_50hertz,DE_LU,DE_amprion,...,PT,RO,RS,SE,SE_1,SE_2,SE_3,SE_4,SI,SK
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+00:00,99.060928,93.422840,454.077703,0.0,714.497120,493.560911,214.625863,290.621266,NaN,357.286846,...,143.119460,383.563632,889.821559,17.171539,0.0,0.0,0.0,0.0,277.651940,161.938148
2018-01-01 01:00:00+00:00,98.561690,98.283497,454.254734,0.0,706.883241,490.276316,213.268384,288.311564,NaN,355.281864,...,147.881433,374.767251,913.739034,17.343350,0.0,0.0,0.0,0.0,272.713457,160.662352
2018-01-01 02:00:00+00:00,105.430789,102.664052,455.118326,0.0,669.643451,486.794095,209.950251,291.409005,NaN,356.630734,...,138.724115,393.938711,920.301544,16.128131,0.0,0.0,0.0,0.0,252.982899,154.836300
2018-01-01 03:00:00+00:00,111.015803,102.033183,454.628404,0.0,668.807796,489.773871,210.666025,287.492822,NaN,359.152793,...,144.387562,403.867663,900.826710,16.175121,0.0,0.0,0.0,0.0,260.642467,144.766345
2018-01-01 04:00:00+00:00,109.723244,102.603786,456.016339,0.0,701.969841,496.084458,208.674903,286.773601,NaN,356.728268,...,144.872729,408.137775,917.659813,16.007878,0.0,0.0,0.0,0.0,262.779394,147.753031


In [16]:
mean_Carbon_intensitiy_entsoe_gen_type = CO2_emission.sum() / Generation.sum()

In [17]:
Carbon_intensitiy_entsoe_gen_type_mean = Carbon_intensitiy_entsoe_gen_type.copy()


In [18]:
for i in Carbon_intensitiy_entsoe_gen_type_mean.columns:
    Carbon_intensitiy_entsoe_gen_type_mean[i] = mean_Carbon_intensitiy_entsoe_gen_type[i]

In [19]:
Carbon_intensitiy_entsoe_gen_type_mean.head()

region,AT,BE,BG,CH,CY,CZ,DE,DE_50hertz,DE_LU,DE_amprion,...,PT,RO,RS,SE,SE_1,SE_2,SE_3,SE_4,SI,SK
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+00:00,140.604427,170.88133,495.5171,0.0,720.0,574.914747,460.819887,640.550396,449.453204,640.323675,...,304.870875,334.159709,852.093782,13.356164,0.0,0.0,0.0,0.0,333.631347,164.919239
2018-01-01 01:00:00+00:00,140.604427,170.88133,495.5171,0.0,720.0,574.914747,460.819887,640.550396,449.453204,640.323675,...,304.870875,334.159709,852.093782,13.356164,0.0,0.0,0.0,0.0,333.631347,164.919239
2018-01-01 02:00:00+00:00,140.604427,170.88133,495.5171,0.0,720.0,574.914747,460.819887,640.550396,449.453204,640.323675,...,304.870875,334.159709,852.093782,13.356164,0.0,0.0,0.0,0.0,333.631347,164.919239
2018-01-01 03:00:00+00:00,140.604427,170.88133,495.5171,0.0,720.0,574.914747,460.819887,640.550396,449.453204,640.323675,...,304.870875,334.159709,852.093782,13.356164,0.0,0.0,0.0,0.0,333.631347,164.919239
2018-01-01 04:00:00+00:00,140.604427,170.88133,495.5171,0.0,720.0,574.914747,460.819887,640.550396,449.453204,640.323675,...,304.870875,334.159709,852.093782,13.356164,0.0,0.0,0.0,0.0,333.631347,164.919239


## Plots

# ENTSO-E Power Stats generation per technology CO2 Signal

In [20]:
MDV = pd.read_csv(input_directory_path + '/MDV_table.csv', index_col=[0], header=[0, 1], parse_dates=True)

In [21]:
MDV.loc['2018'].head(5)

Country                 AL                                             \
ProductionTypeName biomass  gas hard_coal       hydro lignite nuclear   
index                                                                   
2018-01-01             0.0  0.0       0.0  274.666667     0.0     0.0   
2018-02-01             0.0  0.0       0.0  331.000000     0.0     0.0   
2018-03-01             0.0  0.0       0.0  406.000000     0.0     0.0   
2018-04-01             0.0  0.0       0.0  372.666667     0.0     0.0   
2018-05-01             0.0  0.0       0.0  317.666667     0.0     0.0   

Country                                                      ...        TR  \
ProductionTypeName other_fossil other_renewable solar waste  ... hard_coal   
index                                                        ...             
2018-01-01                  0.0             0.0   0.0   0.0  ...   6112.94   
2018-02-01                  0.0             0.0   0.0   0.0  ...   5643.05   
2018-03-01                  0.0             0.0   0.0   0.0  ...   4826.60   
2018-04-01                  0.0             0.0   0.0   0.0  ...   4138.59   
2018-05-01                  0.0             0.0   0.0   0.0  ...   5732.74   

Country                                                                        \
ProductionTypeName        hydro  lignite nuclear other_fossil other_renewable   
index                                                                           
2018-01-01          1508.030000  3611.22     0.0       25.668      199.326667   
2018-02-01          1195.396667  3511.82     0.0       20.726      175.790000   
2018-03-01          2084.640000  3781.64     0.0       19.966      193.436667   
2018-04-01          2109.743333  3629.94     0.0       22.210      184.626667   
2018-05-01          2160.496667  3790.46     0.0       26.972      180.926667   

Country                                                      
ProductionTypeName   solar waste wind_offshore wind_onshore  
index                                                        
2018-01-01          241.15   0.0           0.0      1811.56  
2018-02-01          349.37   0.0           0.0      1673.63  
2018-03-01          381.19   0.0           0.0      2045.82  
2018-04-01          710.82   0.0           0.0      1164.12  
2018-05-01          730.28   0.0           0.0      1217.68  

[5 rows x 432 columns]

In [22]:
# convert to MWh
MDV = MDV * 1000

In [23]:
# Create a dataframe with the EF for each country and technology. If no specific EF is available, we use the default EF .
EF_multi = {}

for i in MDV.columns:
    if i in EF:
        EF_multi[i] = EF[i]
    elif i[1] in EF_mean:
        EF_multi[i] = EF_mean[i[1]]
    else:
        EF_multi[i] = EF_default[i[1]]

In [24]:
# calculate emissions based on EF and generation per type
CO2_emission_MDV = MDV * EF_multi

In [25]:
# From the previously calculated emissions, an emission signal (g CO2 / kWh) can now be calculated for each country.
# Since emissions and generation are available per technology, they must first be grouped and summed. 
CO2_emission_MDV = (CO2_emission_MDV * 1000000).groupby(level=0, axis=1).sum()
Generation_MDV = (MDV * 1000).groupby(level=0, axis=1).sum()

Carbon_intensitiy_MDV = CO2_emission_MDV.div(Generation_MDV)

In [26]:
#take only year 2018
Carbon_intensitiy_MDV = Carbon_intensitiy_MDV.loc['2018']

In [27]:


mean_Carbon_intensitiy_MDV = CO2_emission_MDV.loc['2018'].sum() / Generation_MDV.loc['2018'].sum()

In [28]:
MDV['DE'].sum().sum()

2090735376.6666665

# Compair Bottom Up and Top down

In [29]:
CI_MAP_AP_without_heat_top_down = pd.read_csv(os.path.join(processed_directory_path,'CI_MAP_AP_without_heat_top_down.csv'), index_col=0)
CI_MAP_AP_CHP_heat_top_down = pd.read_csv(os.path.join(processed_directory_path,'CI_MAP_AP_CHP_heat_top_down.csv'), index_col=0)


#CI_top_down.rename(columns={'UK':'GB'}, inplace = True)

In [30]:
CI_compar = pd.DataFrame()
CI_compar.index.rename('country',inplace=True)

In [31]:
CI_compar.index

Index([], dtype='object', name='country')

In [32]:
CI_compar['CI_MAP_AP_without_heat_top_down'] = CI_MAP_AP_without_heat_top_down.loc[2018].filter(mean_Carbon_intensitiy_entsoe_gen_type.index.to_list())

In [33]:
CI_compar['CI_MAP_AP_CHP_heat_top_down'] = CI_MAP_AP_CHP_heat_top_down.loc[2018].filter(mean_Carbon_intensitiy_entsoe_gen_type.index.to_list())

In [34]:
CI_compar['CI_bottom_up'] = mean_Carbon_intensitiy_entsoe_gen_type.filter(CI_compar.index.to_list())
CI_compar['diff_bottom_up_vs_CI_MAP_AP_without_heat_top_down'] =CI_compar['CI_bottom_up'] -  CI_compar['CI_MAP_AP_without_heat_top_down']
CI_compar['diff_pro_1'] = round((100 / CI_compar['CI_MAP_AP_without_heat_top_down']) * CI_compar['diff_bottom_up_vs_CI_MAP_AP_without_heat_top_down'])

CI_compar['diff_bottom_up_vs_CI_MAP_AP_CHP_heat_top_down'] =CI_compar['CI_bottom_up'] -  CI_compar['CI_MAP_AP_CHP_heat_top_down']
CI_compar['diff_pro_2'] = round((100 / CI_compar['CI_MAP_AP_CHP_heat_top_down']) * CI_compar['diff_bottom_up_vs_CI_MAP_AP_CHP_heat_top_down'])


In [35]:
round(CI_compar, 2).sort_values(by='diff_pro_1')

,CI_MAP_AP_without_heat_top_down,CI_MAP_AP_CHP_heat_top_down,CI_bottom_up,diff_bottom_up_vs_CI_MAP_AP_without_heat_top_down,diff_pro_1,diff_bottom_up_vs_CI_MAP_AP_CHP_heat_top_down,diff_pro_2
country,,,,,,,
SE,39.53,12.50,13.36,-26.18,-66.0,0.86,7.0
LT,226.80,56.82,136.42,-90.39,-40.0,79.60,140.0
FI,229.30,108.69,152.77,-76.52,-33.0,44.08,41.0
EE,936.41,886.49,647.06,-289.35,-31.0,-239.43,-27.0
BE,241.20,205.45,170.88,-70.31,-29.0,-34.57,-17.0
NL,535.85,437.55,418.39,-117.46,-22.0,-19.16,-4.0
FR,51.14,53.18,39.69,-11.45,-22.0,-13.48,-25.0
SK,178.54,135.58,164.92,-13.63,-8.0,29.34,22.0
PL,927.70,783.30,855.04,-72.66,-8.0,71.74,9.0


In [36]:
round(CI_compar, 2).to_csv(output_directory_path + '/CI_table.csv')

In [37]:



CI_compar['CI_MDV'] = mean_Carbon_intensitiy_MDV
CI_compar['CI_FACT'] = mean_Carbon_intensitiy_FACT

CI_compar['diff_MDV'] =CI_compar['CI_MDV'] -  CI_compar['CI_top_down_ref']
CI_compar['diff_FACT'] =CI_compar['CI_FACT'] -  CI_compar['CI_top_down_ref']


CI_compar['diff_MDV_pro'] = round((100 / CI_compar['CI_top_down_ref']) * CI_compar['diff_MDV'])

CI_compar['diff_FACT_pro'] = round((100 / CI_compar['CI_top_down_ref']) * CI_compar['diff_FACT'])


NameError: name 'mean_Carbon_intensitiy_FACT' is not defined

In [ ]:
entsoe_gen_type['SE'].sum()/1000

In [ ]:
mean_Carbon_intensitiy_entsoe_gen_type['SE'].sum()/1000

In [ ]:
CI_compar
#gründe warum CI_bottom up zu hoch
# DK wärme wird dort viel von Kraftwerken genutzt
# EE hat ganz viel other fossil (wir haben keine EF ausgerechnet)
# LT
# LV
# NO

Information about the data shown. The reference value shown here is not an official value, but can be used as a reference, since it is based on a documented and freely available method. 

In [ ]:
energy_comp = pd.DataFrame()

energy_comp['MDV'] = MDV['2018'].sum()

In [ ]:
energy_comp['entsoe'] = entsoe_gen_type.sum()

In [ ]:
energy_comp['diff'] = energy_comp['MDV'] - energy_comp['entsoe']
energy_comp['missing_pro'] = round((100 / energy_comp['MDV']) * energy_comp['diff'])

In [ ]:
energy_comp.loc['DE']

In [ ]:
ember_CF = pd.read_csv(os.path.join(input_directory_path,'ember_EF.csv'))

# Plotting CI

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
world[world.name == 'Germany']

In [ ]:
import pycountry



In [ ]:
pycountry.countries.search_fuzzy('Norway')

In [ ]:
import numpy as np


def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
        return result[0].alpha_2
    except:
        return np.nan

world["country_code"] = world["name"].apply(lambda country: do_fuzzy_search(country))

In [ ]:
world.set_index('country_code', inplace=True)

In [ ]:
world['CI'] = Carbon_intensitiy_entsoe_gen_type.mean()

In [ ]:
world[world.name == 'France']

In [ ]:
world.plot(column='CI')

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 1)

fig.suptitle("Density difference plot for " + country, fontsize=16)

divider = make_axes_locatable(ax)

cax = divider.append_axes("right", size="5%", pad=0.1)

world.plot(column='CI',
           ax=ax,
           cax=cax,
           legend=True,
           legend_kwds={'label': "Capacity density diff per NUTS region [MW/km²]", 'orientation': "vertical"},
           cmap='Blues',
           edgecolor='k')


ax.text(0.05, 0.05,'Mean Cap. difference ' + str(round(world.CI.mean())) + ' [MW]',\
        ha='left', va='center', transform=ax.transAxes, fontsize=15, color='white',\
        bbox={'facecolor': 'royalblue', 'alpha': 0.5, 'pad': 5})

ax.title.set_text('Density difference plot for capacities in the NUTS regions')


# Export final CO2 signals

In [ ]:
Carbon_intensitiy_entsoe_gen_type.to_csv(processed_directory_path + '/CI_bottom_up_methode.csv')
Carbon_intensitiy_entsoe_gen_type.to_csv(output_directory_path + '/CI_bottom_up_methode.csv')

# Plottting 

In [ ]:
CI_compar.index

In [ ]:
import numpy as np
# Create a dataframe
#value1=np.random.uniform(size=20)
#value2=value1+np.random.uniform(size=20)/4
#df = pd.DataFrame({'group':list(map(chr, range(65, 85))), 'value1':value1 , 'value2':value2 })

CI_compar

# Reorder it following the values of the first value:
ordered_df = CI_compar.sort_values(by='CI_bottom_up')
my_range=range(1,len(CI_compar.index)+1)
 
# The vertical plot is made using the hline function
# I load the seaborn library only to benefit the nice looking feature
import seaborn as sns
plt.hlines(y=my_range, xmin=ordered_df['CI_top_down_ref'], xmax=ordered_df['CI_bottom_up'], color='grey', alpha=0.4)
plt.scatter(ordered_df['CI_top_down_ref'], my_range, color='skyblue', alpha=1, label='CI_top_down_ref')
plt.scatter(ordered_df['CI_bottom_up'], my_range, color='green', alpha=0.4 , label='CI_bottom_up')
plt.legend()
 
# Add title and axis names
plt.yticks(my_range, ordered_df.index)
plt.title("Comparison of the CI_top_down_ref and the CI_bottom_up", loc='left')
plt.xlabel('Value of the variables')
#plt.ylabel(ordered_df.index)

In [ ]:
df